# Customer churn with a classifcation model

**Content:**
1. [Install packages and load data](#1)
1. [Data Exploration](#2)
1. [Data Preparation](#3)
1. [Feature Engineering](#4)
1. [Multivariate Analysis ](#5)
1. [Split into train- and testset](#6)
1. [Train and validate models](#7)
1. [K-fold cross validation](#8)
1. [Advanced Models I: LightGBM](#9)
1. [Advanced Models II: CatBoost](#10)

The code guides you through the process, but contains a number of '...' and '???' bits that you can fill in yourself. Of course you are also free to add other functions and steps to further improve the churn predictions!

<a id="1"></a> 


## 1. Install packages and load data

In [ ]:
# a) Install packages
import pandas as pd       # 'as' := we abbreviate the package for common use
import numpy as np
import seaborn as sns
import random
import os
import math
import matplotlib.pyplot as plt 
import datetime
import collections
import scipy.stats as stats                             # Fit of distribution plot
from sklearn.model_selection import train_test_split    # Split train-/testset     
from sklearn.tree import DecisionTreeClassifier         # Modeling CART Decision Tree  
from sklearn import metrics                             # Performance metrics
from sklearn.metrics import accuracy_score              # Accuracy of a model
from sklearn.metrics import classification_report       # Performance report of a classification model
from sklearn.metrics import f1_score,average_precision_score                    # f1 score of model
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler,TomekLinks
from sklearn.metrics import precision_recall_curve, auc

import graphviz as gv                                   # Needed to graph Decision Tree
import pydotplus                                        # Needed to graph Decision Tree
from IPython.display import Image                       # Needed to graph Decision Tree
from six import StringIO                                # Needed to graph Decision Tree
from sklearn.tree import export_graphviz                # Needed to graph Decision Tree
from sklearn.ensemble import RandomForestClassifier     # Modeling Random Forest
from sklearn.ensemble import AdaBoostClassifier         # Modeling AdaBoost 
from sklearn.ensemble import GradientBoostingClassifier # Modeling XGBoost
from sklearn.model_selection import KFold               # Cross-validation using stratified K-fold
from sklearn.model_selection import RandomizedSearchCV  # Randomized searching through a parameter grid

In [ ]:
# b) Load data

inputdata = pd.read_csv("Churn_Modelling_undersampled.csv")

#    Get an overview of the data
inputdata.head(10)

<a id="2"></a> 


## 2. Data Exploration 

In [ ]:
# a) Get an overview of the data
#    Get the number of rows and columns
print('(nrow, ncol):', inputdata.shape)     

#    Show a brief summary of the numeric variables
inputdata.describe()                        # min/max, count, mean, std and percentiles

In [ ]:
# b) Check data type of each variable
print(pd.DataFrame(inputdata.dtypes, columns=['Datatype']))

In [ ]:
# c) Get an overview of the NULLS in the dataset
nulls = pd.DataFrame(inputdata.isnull().sum(), columns=['# NULLS'])        # Number of NULLS 

lst={}                                                                     # Number of NULLS as a percentage
for col in inputdata.columns:                                       
    lst[col]=np.sum(inputdata.loc[:,col].isnull())/len(inputdata.loc[:,col])
percNulls = pd.DataFrame(pd.Series(lst), columns=['% NULLS'])

print(pd.concat([nulls, percNulls], axis=1))

In [ ]:
# analyze target variable
target = '???'

inputdata[target].value_counts()

In [ ]:
# e) Analyze categorical variables
inputdata['...'].value_counts()

In [ ]:
# f) Analyze distribution of continuous variable
variable_analyzed = '???'

plt.figure(figsize=(15,8))
plt.hist((inputdata[variable_analyzed]), bins=250, color = 'blue', edgecolor = 'blue')
plt.title('Distribution of {}'.format(variable_analyzed))
plt.xlabel(variable_analyzed)
plt.ylabel('#')

<a id="3"></a> 


## 3. Data Preparation

In [ ]:
print(inputdata.shape)

In [ ]:
# a) Drop outliers (where features have impossibly high or impossibly low values)

inputdata = inputdata[~(inputdata['Age']>=.....)] # What would be a logical cut-off?
inputdata = inputdata[~(inputdata['???']<.....)] # What other features have values that shouldn't exist?

In [ ]:
# b) Deal with the missing data. What is logical to fill the missing data with?
inputdata.NumOfProducts = inputdata.NumOfProducts.fillna(...)

<a id="4"></a> 


## 4. Feature Engineering

In [ ]:
# a) Creating new variables

#inputdata['...'][<condition> = '...'
#inputdata['...'] = inputdata['...'] * inputdata['...']

In [ ]:
# b) Dummmify the categorical variables
categoricals = ['...' 
               ]
inputdata = pd.get_dummies(inputdata, columns=categoricals)

<a id="5"></a> 


## 5. Multivariate Analysis 

In [ ]:
inputdata.columns.values


In [ ]:
# a) Create barplot to see the relation between the dummy-variables and target variable
dummies = ['...'
          ]


plt.figure(figsize=(20,20))
for i, column in enumerate(dummies):
    plt.subplot(math.ceil(len(dummies)/3), 3, i+1)
    sns.barplot(x=column, y=target, data=inputdata, palette='Blues')

In [ ]:
#  b) Show correlation matrix to look for mutual correlations
columns = ['...'
          ]

correlation = inputdata[columns]
corrmat = correlation.corr().round(2)
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True, annot=True, cmap='RdBu_r')

<a id="6"></a> 


## 6. Split into train- and testset

In [ ]:
# a) Split the data. If you do not know what to fill in, google the function (add scikit in the search)
data_train, data_test  = train_test_split(...)

In [ ]:
# b) convert all column names to strings
data_train.columns = data_train.columns.astype(str)
data_test.columns = data_test.columns.astype(str)

<a id="7"></a> 


## 7. Train and validate models

<a id="5a"></a> 
### A) Decision Tree CART

In [ ]:
# get all variable names
data_train.columns.values


In [ ]:
# Define X and y 
X_variables = ['...']           # Adjust to your needs
y_variable = '...'

X_train = data_train.loc[:, X_variables]
y_train = data_train[y_variable]
X_test = data_test.loc[:, X_variables]
y_test = data_test[y_variable]

In [ ]:
# a) Set model parameters 
#    Note: if Min_bucket is too large, the tree might not branch, if too small, the tree might get to big to interpret
Min_num_splits = ...                            # Minimum number of items to split
Min_bucket     = ...               # Minimum number of items per bucket
Max_depth      = ...        # Maximum depth of final tree (nr of levels)

In [ ]:
# b) Estimate the model 
mytree = DecisionTreeClassifier(max_depth=Max_depth
                                ,min_samples_split=Min_num_splits
                                ,min_samples_leaf=Min_bucket
                                ,criterion="gini"              
                                ,splitter="best"
                                ,random_state=random.seed()
                                )

mytree.fit(X_train, y_train)     # Fit the model over the train set

In [ ]:
# c) Create predictions for the test set
preds_proba = mytree.predict_proba(...)
preds = mytree.predict(...)   # Cut-off point equals 0.5

#    Show the first 5 lines of the prediction probabilities and corresponding prediction
pd.concat([pd.DataFrame(preds_proba, columns=["Prob. 0", "Prob. 1"]), pd.DataFrame(preds, columns=["Prediction"])], axis=1).head()

In [ ]:
#    Calculate the optimal cut-off point
cost_TP = ...
cost_TN = ...
cost_FP = ...
cost_FN = ...
total_cost = math.inf

for i in np.linspace(0,1,100,endpoint=False):
    y_pred = (preds_proba[:,1]>i).astype('int')
    results = metrics.confusion_matrix(y_pred,y_test)
    TN = results[0][0]
    FN = results[0][1]
    FP = results[1][0]
    TP = results[1][1]
    
    # Calculate cutoff-point
    cost = TN*cost_TN + TP*cost_TP + FP*cost_FP + FN*cost_FN
    total_cost = min(total_cost,cost)
    if(total_cost == cost):
        opt_cutoff = i
        
print('Optimal cut-off:', opt_cutoff)

#    Create predictions for the test set according to optimal cutoff point
preds = (preds_proba[:,1] > opt_cutoff).astype('int')

In [ ]:
#    Plot tpr vs 1-fpr
fpr, tpr, t = metrics.roc_curve(y_test, preds_proba[:,1])
i = np.arange(len(tpr)) # index for df
roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr, index = i)
                    ,'1-fpr' : pd.Series(1-fpr, index = i)
                    ,'tf' : pd.Series(tpr - (1-fpr), index = i)
                    ,'thresholds' : pd.Series(t, index = i)})
print(roc.loc[(roc.tf-0).abs().argsort()[:1]])
fig, ax = plt.subplots()
plt.plot(roc['tpr'])
plt.plot(roc['1-fpr'], color = 'red')
plt.title('Receiver operating characteristic')
plt.ylabel('True Positive Rate')
plt.xlabel('1-False Positive Rate')
ax.set_xticklabels([])

In [ ]:
# d) Evaluate results
#    i. Create confusion matrix
print(pd.crosstab(..., ...))

In [ ]:
#    ii. Create classification report
print(classification_report(..., ...))
f1_DT = f1_score(y_test, preds)

In [ ]:
#    iii. Get the feature importances of the tree
importances = mytree.feature_importances_ 
std = np.std([mytree.feature_importances_], axis=0)
indices = np.argsort(importances)[::-1]

importances_features = []
print("Feature ranking:")                    # Print the feature ranking
for f in range(X_train.shape[1]):
    print("Feature %d (%s) %f" % (indices[f], X_variables[indices[f]], importances[indices[f]]))
    importances_features.append(X_variables[indices[f]])

plt.figure(figsize=(7.5,5))
plt.figure()                                 # Plot the feature importances 
plt.bar(range(X_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.title("Feature importances")
plt.ylabel("Importance in terms of decreasing the weighted impurity")
plt.xlabel("Feature")
plt.xticks(range(X_train.shape[1]), importances_features, rotation = 30)
plt.xlim([-1, X_train.shape[1]])
plt.show()

In [ ]:
#    iv. Create ROC curve
fpr, tpr, t = metrics.roc_curve(y_test, preds_proba[:,1])

#     v. Calculate AUC
CART_roc_auc_tree = metrics.auc(fpr, tpr)

plt.figure(figsize=(10,6))
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')      # Plot results
plt.plot(fpr, tpr, lw=2, alpha=0.3, label='Mean ROC Decision Tree CART (AUC = %0.2f)' % (CART_roc_auc_tree))
plt.title('ROC curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# e) Visualize Decision Tree
dot_data = StringIO()
export_graphviz(mytree, out_file=dot_data,           # mytree := name of your decision treee
                filled=True, rounded=True,
                feature_names=X_variables,
                special_characters=True)
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

<a id="5c"></a> 
### B) Random Forest 

In [ ]:
# Define X and y 
X_variables = ['...']  # Adjust to your needs

X_train = data_train.loc[:, X_variables]
y_train = data_train[y_variable]
X_test = data_test.loc[:, X_variables]
y_test = data_test[y_variable]

In [ ]:
# a) Set model parameters 
#     Note: You can tweak the size of your forest by changing 'N_trees', but note computation time increases with size
#     Note: if Min_bucket is too large, the trees might not branch
N_trees        = ...                           # Number of trees that are estimated
Min_num_splits = ...                            # Minimum number of items to split    
Min_bucket     = math.floor(Min_num_splits/3)  # Minimum number of items per bucket
Max_depth      = ...                            # Maximum depth of each tree (nr of levels)

In [ ]:
# b) Estimate the model
forest = RandomForestClassifier(n_estimators = N_trees
                                ,criterion = "gini"            
                                ,max_depth = Max_depth
                                ,min_samples_split = Min_num_splits
                                ,min_samples_leaf = Min_bucket
                                ,random_state = random.seed()
                                )

forest.fit(X_train, y_train)   # Fit the model over the train set

In [ ]:
# c) Create predictions for the test set
preds_proba = forest.predict_proba(...)
preds = forest.predict(...)   # Cut-off point equals 0.5

#    Show the first 5 lines of the prediction probabilities and corresponding prediction
pd.concat([pd.DataFrame(preds_proba, columns=["Prob. 0", "Prob. 1"]), pd.DataFrame(preds, columns=["Prediction"])], axis=1).head()

In [ ]:
# d) Evaluate results
#    i. Create confusion matrix
print(pd.crosstab(..., ...))

In [ ]:
#    ii. Create classification report
print(classification_report(..., ...))
f1_RF = f1_score(..., ...)


In [ ]:
#    iii. Get the feature importances of the forest
importances = forest.feature_importances_ 
std = np.std([forest.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

importances_features = []
print("Feature ranking:")                    # Print the feature ranking
for f in range(X_train.shape[1]):
    print("Feature %d (%s) %f" % (indices[f], X_variables[indices[f]], importances[indices[f]]))
    importances_features.append(X_variables[indices[f]])

plt.figure(figsize=(7.5,5))
plt.figure()                                 # Plot the feature importances 
plt.bar(range(X_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.title("Feature importances")
plt.ylabel("Importance in terms of decreasing the weighted impurity")
plt.xlabel("Feature")
plt.xticks(range(X_train.shape[1]), importances_features, rotation = 30)
plt.xlim([-1, X_train.shape[1]])
plt.show()

In [ ]:
#    iv. Create ROC curve
fpr, tpr, t = metrics.roc_curve(y_test, preds_proba[:,1])

#     v. Calculate AUC
RF_roc_auc_tree = metrics.auc(fpr, tpr)

plt.figure(figsize=(10,6))
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')      # Plot results
plt.plot(fpr, tpr, lw=2, alpha=0.3, label='Mean ROC Decision Tree RF (AUC = %0.2f)' % (RF_roc_auc_tree))
plt.title('ROC curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc="lower right")
plt.show()

<a id="5d"></a> 
### C) ADABOOST (Adaptive Boosting Model)

In [ ]:
# Define X and y 
X_variables = ['...']   # Adjust to your needs

X_train = data_train.loc[:, X_variables]
y_train = data_train[y_variable]
X_test = data_test.loc[:, X_variables]
y_test = data_test[y_variable]

In [ ]:
# a) Set model parameters 
#     Note: You can tweak the size of your boost by changing 'N_trees', but note computation time increases with size
#     Note: if Min_bucket is too large, the trees might not branch
N_trees        = ...                          # Number of trees that are estimated
Min_num_splits = ...                            # Minimum number of items to split    
Min_bucket     = math.floor(Min_num_splits/3)  # Minimum number of items per bucket
Max_depth      = ...                             # Maximum depth of each tree (nr of levels)
Learning_rate  = ...                           # The learning rate

In [ ]:
# b) Estimate model
adaptboost = AdaBoostClassifier(n_estimators=N_trees
                                ,base_estimator=DecisionTreeClassifier(max_depth = Max_depth
                                                                       ,min_samples_split = Min_num_splits
                                                                       ,min_samples_leaf = Min_bucket
                                                                       ,criterion = "gini"              
                                                                       ,splitter = "best"
                                                                       )
                                ,learning_rate=Learning_rate
                               )

adaptboost.fit(X_train, y_train)      # Fit the model over the train set

In [ ]:
# c) Create predictions for the test set
preds_proba = adaptboost.predict_proba(...)
preds = adaptboost.predict(...)   # Cut-off point equals 0.5

#    Show the first 5 lines of the prediction probabilities and corresponding prediction
pd.concat([pd.DataFrame(preds_proba, columns=["Prob. 0", "Prob. 1"]), pd.DataFrame(preds, columns=["Prediction"])], axis=1).head()

In [ ]:
# d) Evaluate results
#    i. Create confusion matrix
print(pd.crosstab(..., ...))

In [ ]:
#    ii. Create classification report
print(classification_report(..., ...))
f1_ADA = f1_score(..., ...)

In [ ]:
#    iii. Get the feature importances of the Adaboost
importances = adaptboost.feature_importances_ 
std = np.std([adaptboost.feature_importances_ for tree in adaptboost.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

importances_features = []
print("Feature ranking:")                    # Print the feature ranking
for f in range(X_train.shape[1]):
    print("Feature %d (%s) %f" % (indices[f], X_variables[indices[f]], importances[indices[f]]))
    importances_features.append(X_variables[indices[f]])

plt.figure(figsize=(7.5,5))
plt.figure()                                 # Plot the feature importances 
plt.bar(range(X_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.title("Feature importances")
plt.ylabel("Importance in terms of decreasing the weighted impurity")
plt.xlabel("Feature")
plt.xticks(range(X_train.shape[1]), importances_features, rotation = 30)
plt.xlim([-1, X_train.shape[1]])
plt.show()

In [ ]:
#    iv. Create ROC curve
fpr, tpr, t = metrics.roc_curve(y_test, preds_proba[:,1])

#     v. Calculate AUC
ADA_roc_auc_tree = metrics.auc(fpr, tpr)

plt.figure(figsize=(10,6))
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')      # Plot results
plt.plot(fpr, tpr, lw=2, alpha=0.3, label='Mean ROC Decision Tree ADABOOST (AUC = %0.2f)' % (ADA_roc_auc_tree))
plt.title('ROC curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# e) Plot error with respect to number of estimators (trees)
trained_adaptboost = adaptboost.fit(X_train, y_train)  

real_test_errors = []
for real_test_predict in trained_adaptboost.staged_predict(X_test):
    real_test_errors.append(1. - accuracy_score(real_test_predict, np.ravel(y_test)))
n_trees_real = len(trained_adaptboost)
real_estimator_errors = trained_adaptboost.estimator_errors_[:n_trees_real]
plt.figure(figsize=(15, 5))

plt.plot(range(1, n_trees_real + 1),
         real_test_errors, c='black',
         linestyle='dashed')
plt.ylim(min(real_test_errors), max(real_test_errors)+0.1)
plt.title('Error with respect to the number of trees')
plt.ylabel('Test Error')
plt.xlabel('Number of estimators')

<a id="5e"></a> 
### D) XGBoost

In [ ]:
# Define X and y 
X_variables = ['...']   # Adjust to your needs

X_train = data_train.loc[:, X_variables]
y_train = data_train[y_variable]
X_test = data_test.loc[:, X_variables]
y_test = data_test[y_variable]

In [ ]:
# a) Set model parameters 
N_trees        = ...        # Number of trees that are estimated
Max_depth      = ...        # Maximum depth of each tree (nr of levels)
Learning_rate  = ...        # The learning rate ('eta')
Min_bucket     = ...        # Minimum number of items per bucket
Subsample      = ...        # Subsample ratio of the training instance
Verbose        = ...        # Whether to print messages while running boosting

In [ ]:
# b) Estimate model
XGB = GradientBoostingClassifier( random_state=12,
     n_estimators=N_trees
                         ,max_depth=Max_depth 
                         ,learning_rate=Learning_rate
                         ,min_samples_leaf=Min_bucket
                         ,subsample=Subsample
                         ,verbose=Verbose
                      )
XGB.fit(X_train, y_train)      # Fit the model over the train set

In [ ]:
# c) Create predictions for the test set
preds_proba = XGB.predict_proba(...)
preds = XGB.predict(...)   # Cut-off point equals 0.5

#    Show the first 5 lines of the prediction probabilities and corresponding prediction
#pd.concat([pd.DataFrame(preds_proba, columns=["Prob. 0", "Prob. 1"]), pd.DataFrame(preds, columns=["Prediction"])], axis=1).head()

In [ ]:
# d) Evaluate results
#    i. Create confusion matrix
print(pd.crosstab(..., ...))

In [ ]:
#    ii. Create classification report
print(classification_report(..., ...))
f1_XGB = f1_score(..., ...)

In [ ]:
#    iii. Get the feature importances of the XGBoost
importances = XGB.feature_importances_ 
std = np.std([XGB.feature_importances_], axis=0)
indices = np.argsort(importances)[::-1]

importances_features = []
print("Feature ranking:")                    # Print the feature ranking
for f in range(X_train.shape[1]):
    print("Feature %d (%s) %f" % (indices[f], X_variables[indices[f]], importances[indices[f]]))
    importances_features.append(X_variables[indices[f]])

plt.figure(figsize=(7.5,5))
plt.figure()                                 # Plot the feature importances 
plt.bar(range(X_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.title("Feature importances")
plt.ylabel("Importance in terms of decreasing the weighted impurity")
plt.xlabel("Feature")
plt.xticks(range(X_train.shape[1]), importances_features, rotation = 30)
plt.xlim([-1, X_train.shape[1]])
plt.show()

In [ ]:
# Create ROC curve
fpr, tpr, t = metrics.roc_curve(y_test, preds_proba[:,1])

# Calculate AUC
XGB_roc_auc_tree = metrics.auc(fpr, tpr)

plt.figure(figsize=(10,6))
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')      # Plot results
plt.plot(fpr, tpr, lw=2, alpha=0.3, label='Mean ROC Decision Tree XGBoost (AUC = %0.2f)' % (XGB_roc_auc_tree))
plt.title('ROC curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc="lower right")
plt.show()

### Compare models

In [ ]:
print('AUC Decision Tree:', CART_roc_auc_tree)
print('AUC Random Forest:', RF_roc_auc_tree)
print('AUC Adaptive Boosting:', ADA_roc_auc_tree)
print('AUC XGBoost:', roc_auc_xgboost)

<a id="8"></a> 


## 8. K-fold cross validation

The code below iterates through several data balancing methods and several (randomly selected) hyperparameter configurations for each of the four model techniques. <br>

The code below uses a Random Search to save time. However, you can also replace it by a full Grid Search to find even better results! See https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html for an example.

In [ ]:
# Define X and y
X_variables = ['...', '...'] # Adjust to your choice
y_variable = 'Exited'

X_train = data_train.loc[:, X_variables]
y_train = data_train[y_variable]
X_test = data_test.loc[:, X_variables]
y_test = data_test[y_variable]

# Sampling methods
desiredPercentage = ??? # Decide the percentage that you want in over/under sampling
desiredFraction = desiredPercentage/(1-desiredPercentage)

ranOS = RandomOverSampler(sampling_strategy=desiredFraction, random_state=1) # Oversampling
ranUS = RandomUnderSampler(sampling_strategy=desiredFraction, random_state=1) # Undersampling
smt = SMOTE(k_neighbors=2) # SMOTE (bonus: tweak the number of neighbors to maximize results)
TL = TomekLinks('not majority') # TomekLinks

sampling_methods=[None, ranOS, ranUS, smt, TL]
sampling_method_names=['None', 'Oversampling', 'Undersampling', 'SMOTe', 'Tomek Links']

# Define parameter grid. Note: some parameters apply to all model techniques, some only to one or some of them.
# Fill in appropriate values at the ... yourself.
# If you want some ideas for what values might work, you can look up the documentation or other examples online!
criterion         = ["gini"]
learning_rate     = [0.05, 0.1, 0.2, 0.5, 1.0]
max_depth         = [...]
max_features      = [None, 'sqrt', 'log2', 0.5, 0.2, ...]
min_samples_leaf  = [...]
min_samples_split = [...]
n_estimators      = [...]
splitter          = ["best"]
subsample         = [...]

n_iterations = 2 # Number of iterations to use in each random search.
# Increasing this value will most likely increase model performance, but certainly also increase the time it takes!

aucs = pd.DataFrame(index=range(0, len(sampling_methods)), columns=['Sampling method', 'Decision Tree','Random Forest','Adaboost','XGBoost'])
for i in range(len(sampling_methods)):
    
    j='Sampling method'
    sampling_method = sampling_methods[i]
    sampling_method_name = sampling_method_names[i]
    if sampling_method == None:
        X_train_balanced, y_train_balanced = X_train, y_train
    else:
        X_train_balanced, y_train_balanced = sampling_method.fit_resample(X_train, y_train) # Use sampling method
    aucs.loc[i,j] = sampling_method_name

    # a) Decision Tree CART
    j='Decision Tree'
    print(sampling_method_name, '-', j)
    dt_params = dict(
        criterion         = criterion         ,
        max_depth         = max_depth         ,
        max_features      = max_features      ,
        min_samples_leaf  = min_samples_leaf  ,
        min_samples_split = min_samples_split ,
        splitter          = splitter          
    )
    dt_search = RandomizedSearchCV(DecisionTreeClassifier(), dt_params, n_iter=n_iterations, verbose=False)
    dt_search = dt_search.fit(X_train_balanced, y_train_balanced)
    dt_model = dt_search.best_estimator_
    preds_proba = dt_model.predict_proba(X_test)
    aucs.loc[i,j] = average_precision_score(y_test, preds_proba[:,1])           # Calculate AP
    
    # b) Random Forest
    j='Random Forest'
    print(sampling_method_name, '-', j)
    rf_params = dict(
        criterion         = criterion         ,
        max_depth         = max_depth         ,
        max_features      = max_features      ,
        min_samples_leaf  = min_samples_leaf  ,
        min_samples_split = min_samples_split ,
        n_estimators      = n_estimators      
    )
    rf_search = RandomizedSearchCV(RandomForestClassifier(), rf_params, n_iter=n_iterations, verbose=False)
    rf_search = rf_search.fit(X_train_balanced, y_train_balanced)
    rf_model = rf_search.best_estimator_
    preds_proba = rf_model.predict_proba(X_test)
    aucs.loc[i,j] = average_precision_score(y_test, preds_proba[:,1])           # Calculate AP
    
    # c) Adaboost
    j='Adaboost'
    print(sampling_method_name, '-', j)
    ada_params = dict(
        base_estimator__criterion         = criterion         ,
        learning_rate                     = learning_rate     ,
        base_estimator__max_depth         = max_depth         ,
        base_estimator__max_features      = max_features      ,
        base_estimator__min_samples_leaf  = min_samples_leaf  ,
        base_estimator__min_samples_split = min_samples_split ,
        n_estimators                      = n_estimators      ,
        base_estimator__splitter          = splitter          
    )
    ada_search = RandomizedSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), ada_params, n_iter=n_iterations, verbose=False)
    ada_search = ada_search.fit(X_train_balanced, y_train_balanced)
    ada_model = ada_search.best_estimator_
    preds_proba = ada_model.predict_proba(X_test)
    aucs.loc[i,j] = metrics.auc(fpr, tpr)           # Calculate AUC
    
    # d) XGBoost
    j='XGBoost'
    print(sampling_method_name, '-', j)
    xgb_params = dict(
        learning_rate     = learning_rate     ,
        max_depth         = max_depth         ,
        max_features      = max_features      ,
        min_samples_leaf  = min_samples_leaf  ,
        min_samples_split = min_samples_split ,
        n_estimators      = n_estimators      ,
        subsample         = subsample         
    )
    xgb_search = RandomizedSearchCV(GradientBoostingClassifier(), xgb_params, n_iter=n_iterations, verbose=False)
    xgb_search = xgb_search.fit(X_train_balanced, y_train_balanced)
    xgb_model = xgb_search.best_estimator_
    preds_proba = xgb_model.predict_proba(X_test)
    aucs.loc[i,j] = metrics.auc(fpr, tpr)           # Calculate AUC

aucs

In [ ]:
# Get feature importances from your model of choice
model = ... # Choose between dt_model, rf_model, ada_model, xgb_model

importances = model.feature_importances_ 
std = np.std([importances], axis=0)
indices = np.argsort(importances)[::-1]

importances_features = []
print("Feature ranking:")                    # Print the feature ranking
for f in range(X_train.shape[1]):
    print("Feature %d (%s) %f" % (indices[f], X_variables[indices[f]], importances[indices[f]]))
    importances_features.append(X_variables[indices[f]])

plt.figure(figsize=(7.5,5))
plt.figure()                                 # Plot the feature importances 
plt.bar(range(X_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.title("Feature importances")
plt.ylabel("Importance in terms of decreasing the weighted impurity")
plt.xlabel("Feature")
plt.xticks(range(X_train.shape[1]), importances_features, rotation = 30)
plt.xlim([-1, X_train.shape[1]])
plt.show()

<a id="9"></a> 


## 9. Advanced Models I: LightGBM

In [ ]:
# Import relevant package
import lightgbm as lgb
import time

# Define X and y 
# X_variables = [''                 # ADJUST VARIABLES TO THOSE YOU WISH TO INCLUDE

#               ]
# y_variable = ''

X_train = data_train.loc[:, X_variables]
y_train = data_train[y_variable]
X_test = data_test.loc[:, X_variables]
y_test = data_test[y_variable]

print(X_variables)

# a) Set model parameters 
N_trees           = 100                           # Number of trees that are estimated
Max_depth         = -1                            # Maximum depth of each tree (nr of levels); -1 means no limit
Learning_rate     = 0.1                           # The learning rate ('eta')
Min_child_samples = 10                            # Minimum number of items per bucket
Min_child_weight  = 0.001                         # Minimum sum of instance weight (hessian) needed in a child (leaf).
Subsample         = 1                             # Subsample ratio of the training instance
Silent            = 1                             # Whether to print messages while running boosting
Num_leaves        = 70                            # Maximum number of leafs per tree

# Note: there are way more hyperparameters to tune. You can find them well-explained here: 
# https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
# The values above are not tuned and optimized yet

# b) Estimate model
lgb_model = lgb.LGBMClassifier(
                         n_estimators=N_trees
                        ,max_depth=Max_depth 
                        ,learning_rate=Learning_rate
                        ,min_child_weight=Min_child_weight
                        ,min_child_samples=Min_child_samples
                        ,subsample=Subsample
                        ,num_leaves=Num_leaves
                        ,silent=Silent
                       )

lgb_model.fit(X_train, y_train)      # Fit the model over the train set

# c) Create predictions for the test set
preds_proba = lgb_model.predict_proba(X_test)
preds = lgb_model.predict(X_test)   # Cut-off point equals 0.5

#    Show the first 5 lines of the prediction probabilities and corresponding prediction
#pd.concat([pd.DataFrame(preds_proba, columns=["Prob. 0", "Prob. 1"]), pd.DataFrame(preds, columns=["Prediction"])], axis=1).head()

# d) Evaluate results
#    i. Create confusion matrix
#print(pd.crosstab(preds, y_test))

#    ii. Create classification report
#print(classification_report(y_test, preds))
f1_lgb_model = f1_score(y_test, preds)

#    iii. Get the feature importances of the lgb_modeloost
importances = lgb_model.feature_importances_ 
std = np.std([lgb_model.feature_importances_], axis=0)
indices = np.argsort(importances)[::-1]

importances_features = []
print("Feature ranking:")                    # Print the feature ranking
for f in range(X_train.shape[1]):
    print("Feature %d (%s) %f" % (indices[f], X_variables[indices[f]], importances[indices[f]]))
    importances_features.append(X_variables[indices[f]])

if False:
    plt.figure(figsize=(7.5,5))
    plt.figure()                                 # Plot the feature importances 
    plt.bar(range(X_train.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.title("Feature importances")
    plt.ylabel("Importance in terms of decreasing the weighted impurity")
    plt.xlabel("Feature")
    plt.xticks(range(X_train.shape[1]), importances_features, rotation = 30)
    plt.xlim([-1, X_train.shape[1]])
    plt.show()

# Create ROC curve
fpr, tpr, t = metrics.roc_curve(y_test, preds_proba[:,1])

# Calculate AUC
lgb_model_roc_auc_tree = metrics.auc(fpr, tpr)

plt.figure(figsize=(10,6))
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')      # Plot results
plt.plot(fpr, tpr, lw=2, alpha=0.3, label='Mean ROC Decision Tree LightGBM (AUC = %0.2f)' % (lgb_model_roc_auc_tree))
plt.title('ROC curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc="lower right")
plt.show()

<a id="10"></a> 


## 10. Advanced Models II: CatBoost

In [ ]:
# Import relevant package
import catboost as cat
import time

# Define X and y 
X_variables = [...]

X_train = data_train.loc[:, X_variables]
y_train = data_train[y_variable]
X_test = data_test.loc[:, X_variables]
y_test = data_test[y_variable]

# Because we use CatBoost, we can leverage its abilitiy to handle categoricals in a clever way
# For that, we need the column indices of the cat_features
cat_features=[...]
# Be aware: you might need to add the non-dummified columns to your dataset again, as those are dropped by default after set_dummies

# a) Set model parameters 
N_trees        = ...                           # Number of trees that are estimated
Max_depth      = ...                           # Maximum depth of each tree (nr of levels)
Learning_rate  = ...                           # The learning rate ('eta')
Subsample      = ...                           # Subsample ratio of the training instance
Silent         = True                          # Whether to print messages while running boosting

# Note: there are way more hyperparameters to tune. You can find them well-explained here: 
# https://catboost.ai/docs/concepts/parameter-tuning.html
# The values above are not tuned and optimized yet

# b) Estimate model
cat_model = cat.CatBoostClassifier(
                        n_estimators=N_trees
                        ,max_depth=Max_depth 
                        ,learning_rate=Learning_rate
                        ,subsample=Subsample
                        ,silent=Silent
                        ,cat_features=... 
                       )
# You can uncomment the cat_features if you don't want to use this. 
# Given that we don't have many categories, this should not matter much. 
# But we leave it up to you to find out what the difference in speed and performance is!

cat_model.fit(X_train, y_train)      # Fit the model over the train set

# c) Create predictions for the test set
preds_proba = cat_model.predict_proba(X_test)
preds = cat_model.predict(X_test)   # Cut-off point equals 0.5

#    Show the first 5 lines of the prediction probabilities and corresponding prediction
#pd.concat([pd.DataFrame(preds_proba, columns=["Prob. 0", "Prob. 1"]), pd.DataFrame(preds, columns=["Prediction"])], axis=1).head()

# d) Evaluate results
#    i. Create confusion matrix
#print(pd.crosstab(preds, y_test))

#    ii. Create classification report
#print(classification_report(y_test, preds))
f1_cat_model = f1_score(y_test, preds)

#    iii. Get the feature importances of the cat_modeloost
importances = cat_model.feature_importances_ 
std = np.std([cat_model.feature_importances_], axis=0)
indices = np.argsort(importances)[::-1]

importances_features = []
print("Feature ranking:")                    # Print the feature ranking
for f in range(X_train.shape[1]):
    print("Feature %d (%s) %f" % (indices[f], X_variables[indices[f]], importances[indices[f]]))
    importances_features.append(X_variables[indices[f]])

if False:
    plt.figure(figsize=(7.5,5))
    plt.figure()                                 # Plot the feature importances 
    plt.bar(range(X_train.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.title("Feature importances")
    plt.ylabel("Importance in terms of decreasing the weighted impurity")
    plt.xlabel("Feature")
    plt.xticks(range(X_train.shape[1]), importances_features, rotation = 30)
    plt.xlim([-1, X_train.shape[1]])
    plt.show()

# Create ROC curve
fpr, tpr, t = metrics.roc_curve(y_test, preds_proba[:,1])

# Calculate AUC
cat_model_roc_auc_tree = metrics.auc(fpr, tpr)

plt.figure(figsize=(10,6))
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')      # Plot results
plt.plot(fpr, tpr, lw=2, alpha=0.3, label='Mean ROC Decision Tree CatBoost (AUC = %0.2f)' % (cat_model_roc_auc_tree))
plt.title('ROC curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc="lower right")
plt.show()